<a href="https://colab.research.google.com/github/burakozturan/bliss/blob/main/BLISS_Lab09_LLMwAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 09: LLMs with APIs - Scaling Up Your Research

**Duration**: 120 minutes | **Prerequisites**: Lab 08 (LLM Capabilities)

## 🎯 Learning Objectives
By the end of this lab, you'll be able to:
- Set up and use API keys securely for philosophical research
- Call APIs to analyze philosophical texts
- Compare different AI models for your research needs
- Engineer effective prompts for better philosophical analysis
- Process multiple texts in batches through APIs
- Import your own data and apply all API skills

## 🤔 Why APIs for Philosophy Research?

**Simple Analogy**: An API is like ordering at a restaurant
- You tell the waiter what you want (your philosophical question)
- The waiter takes it to the kitchen (powerful AI model in the cloud)
- The kitchen prepares your order (processes your text)
- You receive the analysis back

| Aspect | Lab 08 (Local Models) | Lab 09 (APIs) |
|--------|----------------------|----------------|
| **Power** | Limited by your computer | GPT-4, Claude, etc. |
| **Cost** | Free | Small cost per use |
| **Speed** | Depends on your hardware | Usually faster |
| **Privacy** | Completely private | Shared with provider |
| **Capability** | Basic analysis | Sophisticated reasoning |

**Today's Plan**: Learn each API skill step-by-step, one concept at a time.

## Part 1: API Key Setup (15 minutes)

### 📚 What are APIs and API Keys?

**API** = Application Programming Interface
- A way for your code to "talk" to powerful AI models on the internet
- Like calling a really smart philosophy expert on the phone

**API Key** = Your password to use the API
- Proves you're authorized to use the service
- Like showing your library card to check out books

**Why we need API keys**: They let you access much more powerful AI models than what runs on your computer

### 🔐 Security First
**Important**: Never put API keys directly in your code! We'll store them securely.

In [ ]:
# Install required packages
!pip install -q requests pandas openai huggingface_hub
!pip install transformers accelerate datasets sentencepiece -q

from huggingface_hub import InferenceClient
from openai import OpenAI
import requests
import json
import pandas as pd
import time
from google.colab import userdata
import os

print("✅ Setup complete!")

### 🔍 Demonstration: Setting Up HuggingFace API Key

**Follow these steps to get your HuggingFace API key**:

1. Go to [huggingface.co](https://huggingface.co)
2. Sign up (free account)
3. Click your profile → Settings → Access Tokens
4. Create new token → Copy it
5. In Google Colab: Click 🔑 (key icon) on left sidebar
6. Add secret: Name = `HF_API_KEY`, Value = your token

**Watch me set up the secure key loading**:

In [ ]:
# 🔍 DEMONSTRATION: Secure API key loading

def setup_huggingface_api():
    """Securely load HuggingFace API key"""
    try:
        hf_api_key = userdata.get('HF_API_KEY')
        print("✅ HuggingFace API key loaded successfully!")
        print("🔐 Key is hidden for security")
        return hf_api_key
    except:
        print("❌ HuggingFace API key not found.")
        print("Please add it using the steps above.")
        return None

# Test the setup
hf_key = setup_huggingface_api()

if hf_key:
    print("\n🎉 Great! HuggingFace API is ready to use.")
else:
    print("\n⚠️ Please set up your HuggingFace API key first.")

In [ ]:
import os

os.environ["HF_API_KEY"] = "hugginfacekey"
def setup_huggingface_api():
    hf_api_key = os.getenv("HF_API_KEY")
    if hf_api_key:
        print("✅ HuggingFace API key loaded successfully!")
        return hf_api_key
    else:
        print("❌ HuggingFace API key not found.")
        return None
hf_key = setup_huggingface_api()


from huggingface_hub import login
login()


### 🎯 Exercise 1: Set Up OpenRouter API Key

**Your Task**: Follow the same process for OpenRouter

**Steps for OpenRouter**:
1. Go to [openrouter.ai](https://openrouter.ai)
2. Sign up (get $1 free credit)
3. Dashboard → API Keys → Create new key
4. Copy the key
5. In Colab secrets: Add `OPENROUTER_API_KEY`

**Now complete this exercise**:

In [ ]:
# 🎯 YOUR TURN: Set up OpenRouter API key

def setup_openrouter_api():
    """TODO: Complete this function to load OpenRouter API key"""
    try:
        # TODO: Get the OpenRouter API key from userdata
        or_api_key = None  # TODO: Replace None with userdata.get('OPENROUTER_API_KEY')

        print("✅ OpenRouter API key loaded successfully!")
        print("🔐 Key is hidden for security")
        return or_api_key
    except:
        print("❌ OpenRouter API key not found.")
        return None

# TODO: Test your setup and check both keys work
or_key = None  # TODO: Call your function

# TODO: Check if both keys are working
# Complete the if/elif/else logic to check different combinations of hf_key and or_key
# Remember: hf_key was created in the demonstration above

In [ ]:
# Exercise 1 Solution

def setup_openrouter_api():
    """Complete this function to load OpenRouter API key"""
    try:
        # Get the OpenRouter API key from userdata
        or_api_key = userdata.get('OPENROUTER_API_KEY')

        print("✅ OpenRouter API key loaded successfully!")
        print("🔐 Key is hidden for security")
        return or_api_key
    except:
        print("❌ OpenRouter API key not found.")
        print("Please add it using the steps above.")
        return None

# Test your setup
or_key = setup_openrouter_api()

# Check if both keys are working
if hf_key and or_key:
    print("\n🎉 Excellent! Both API keys are ready.")
    print("🚀 You're ready to use powerful AI models!")
elif hf_key:
    print("\n⚠️ HuggingFace ready, but still need OpenRouter key.")
elif or_key:
    print("\n⚠️ OpenRouter ready, but still need HuggingFace key.")
else:
    print("\n❌ Please set up both API keys before continuing.")

print("\n💡 Teaching Note: Students learn secure credential management")

## Part 2: First API Test (15 minutes)

### 📚 How to Call an API

**Basic Process**:
1. Send your philosophical question to the API
2. API processes it through powerful AI model
3. Get back sophisticated analysis

**Like Lab 07 pipelines, but much more powerful!**

### 🔍 Demonstration: First HuggingFace API Call

In [ ]:
# 🔍 DEMONSTRATION: First API call to HuggingFace
from huggingface_hub import InferenceClient

def call_huggingface_api(
    text,
    model="meta-llama/Llama-3.2-1B-Instruct",   # ✅ tiny model WITH chat support
    max_tokens=500
):
    """Call HuggingFace API with philosophical text using chat.completions"""

    if hf_key is None:
        return "❌ HuggingFace API key not available"

    try:
        # Create HF API client
        client = InferenceClient(token=hf_key)

        # Send chat completion request
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": text}],
            max_tokens=max_tokens,     # ✅ you now control answer length
            temperature=0.7
        )

        return completion.choices[0].message["content"]

    except Exception as e:
        return f"❌ HuggingFace Error: {str(e)}"

# Test with a philosophical question
philosophical_question = "What is the difference between knowledge and belief?"

print("🧠 Testing HuggingFace API...")
print(f"Question: {philosophical_question}")
print("\n📡 Sending to API...")

response = call_huggingface_api(philosophical_question)

print("\n✅ Response received:")
print(f"{response}")

print("\n💡 Key Insight: The API gave us a sophisticated philosophical analysis!")

### 🎯 Exercise 2: Your First OpenRouter API Call

**Your Task**: Create an OpenRouter API function and test it with a different philosophical question

**Challenge**: Follow the same pattern as HuggingFace, but for OpenRouter

In [ ]:
# 🎯 YOUR TURN: Create OpenRouter API function

def call_openrouter_api(text, model="x-ai/grok-4.1-fast:free"):
    """ Complete this function to call OpenRouter API"""
    if or_key is None:
        return "❌ OpenRouter API key not available"

    try:
        # : Create OpenAI client for OpenRouter
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=or_key,
        )

        # : Send the philosophical question
        completion = client.chat.completions.create(
            extra_headers={
                "HTTP-Referer": "https://aide-philosophy-research.org",
                "X-Title": "AIDE Philosophy Research",
            },
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": text  # : Use the text parameter
                }
            ]
        )

        # : Return the response content
        return completion.choices[0].message.content
    except Exception as e:
        return f"❌ OpenRouter Error: {str(e)}"


In [ ]:
# TODO: Test with your own philosophical question
your_philosophical_question = "Your question here..."  # TODO: Change this to your actual philosophical question

print("🌐 Testing OpenRouter API...")
print(f"Question: {your_philosophical_question}")
print("\n📡 Sending to API...")

# TODO: Call OpenRouter API with your question
# Hint: Use the call_openrouter_api() function
your_response = None  # TODO: Replace None with the API call

print("\n✅ Response received:")
# TODO: Print your response
print("TODO: Print your response here")

# TODO: Compare with HuggingFace
print("\n🔄 Comparing same question with HuggingFace...")
# TODO: Call HuggingFace API with the same question
# Hint: Use the call_huggingface_api() function
hf_response = None  # TODO: Replace None with the API call

# TODO: Print both responses side by side (first 100 characters)
# Hint: Use [:100] to get first 100 characters
print("TODO: Print HuggingFace preview here")
print("TODO: Print OpenRouter preview here")

print("\n💡 Key Insight: Different APIs can give different perspectives on the same question!")

## Part 3: Model Comparison (20 minutes)

### 📚 Why Compare Different Models?

**Different models have different strengths**:
- Some are better at reasoning
- Some are better at creative thinking  
- Some are free, some cost money
- Some are faster, some are more thorough

**For philosophy research**: You want to find the model that best understands your specific area

### 🔍 Demonstration: Comparing 3 Different Models

In [ ]:
# 🔍 DEMONSTRATION: Compare multiple models on same philosophical problem

def compare_models_demo():
    """Compare 3 different models on the same philosophical question"""

    # The philosophical question to test
    test_question = "Is free will compatible with determinism? Explain briefly."

    # Models to compare
    models_to_test = [
        {
            'name': 'HuggingFace Phi-4',
            'api': 'huggingface',
            'model': 'microsoft/phi-4',
            'cost': 'Free'
        },
        {
            'name': 'OpenRouter Free',
            'api': 'openrouter',
            'model': 'moonshotai/kimi-k2:free',
            'cost': 'Free'
        },
        {
            'name': 'OpenRouter GPT-3.5',
            'api': 'openrouter',
            'model': 'openai/gpt-3.5-turbo',
            'cost': 'Paid (~$0.002/1K tokens)'
        }
    ]

    print("🔍 MODEL COMPARISON DEMONSTRATION")
    print(f"Question: {test_question}")
    print("=" * 60)

    results = []

    for model_info in models_to_test:
        print(f"\n🧠 Testing {model_info['name']} ({model_info['cost']})...")

        # Call the appropriate API
        start_time = time.time()
        if model_info['api'] == 'huggingface':
            response = call_huggingface_api(test_question, model_info['model'])
        else:
            response = call_openrouter_api(test_question, model_info['model'])

        response_time = time.time() - start_time

        # Store result
        result = {
            'model': model_info['name'],
            'response': response,
            'time': response_time,
            'cost': model_info['cost'],
            'length': len(response.split()) if isinstance(response, str) else 0
        }
        results.append(result)

        # Show preview
        print(f"Time: {response_time:.1f}s | Length: {result['length']} words")
        print(f"Response: {response[:150]}...")

        time.sleep(2)  # Be nice to APIs

    # Summary comparison
    print("\n📊 COMPARISON SUMMARY:")
    print("-" * 60)
    for result in results:
        print(f"{result['model']:<20} | {result['time']:.1f}s | {result['length']:3d} words | {result['cost']}")

    print("\n💡 Key Observations:")
    print("• Different models give different perspectives")
    print("• Response length and depth vary")
    print("• Speed differences between models")
    print("• Free vs paid models show different capabilities")

    return results

# Run the demonstration
demo_results = compare_models_demo()

In [ ]:
demo_results

### 🎯 Exercise 3: Compare Models with Your Philosophical Question

**Your Task**: Test the same 3 models with a philosophical question from your own research area

**Goal**: Decide which model works best for your type of philosophical analysis

In [ ]:
# 🎯 YOUR TURN: Compare models with your research question

def your_model_comparison():
    """TODO: Compare models with your own philosophical question"""

    # TODO: Choose a question from your research area
    your_research_question = "Your question here..."  # TODO: Replace with your actual question

    print("🔍 YOUR MODEL COMPARISON")
    print(f"Your Question: {your_research_question}")
    print("=" * 60)

    models = [
        {'name': 'HuggingFace', 'api': 'huggingface'}, # TODO: Replace with your own model
        {'name': 'OpenRouter Free', 'api': 'openrouter', 'model': 'moonshotai/kimi-k2:free'}, # TODO: Replace with your own model
        {'name': 'OpenRouter GPT-3.5', 'api': 'openrouter', 'model': 'openai/gpt-3.5-turbo'} # TODO: Replace with your own model
    ]

    your_results = []

    for i, model_info in enumerate(models):
        print(f"\n🧠 Testing {model_info['name']}...")

        # TODO: Time the response
        start_time = None  # TODO: Get current time using time.time()

        # TODO: Call the appropriate API based on model_info['api']
        if None:  # TODO: Check if api is 'huggingface'
            response = None  # TODO: Call HuggingFace API
        else:
            response = None  # TODO: Call OpenRouter API with model

        # TODO: Calculate response time
        response_time = None  # TODO: Current time minus start_time

        # TODO: Store the result in a dictionary
        result = {
            # TODO: Fill in the dictionary with model name, response, time, and word count
        }
        your_results.append(result)

        # TODO: Show preview (time, word count, first 100 characters)
        print("TODO: Print time and length")
        print("TODO: Print response preview")

        time.sleep(2)

    # TODO: Print results summary table
    print("\n📊 YOUR RESULTS:")
    for result in your_results:
        # TODO: Print formatted results for each model
        print("TODO: Print model comparison line")

    return your_results

# TODO: Run your comparison
my_results = None  # TODO: Call your function

# TODO: Decide which model you prefer
print("\n🎯 My Decision:")
print("For my research, I choose: _____ because _____")

## Part 4: Prompt Engineering (25 minutes)

### 📚 What is Prompt Engineering?

**Prompt Engineering** = The art of asking AI the right questions in the right way

**Why it matters**: The same AI model can give vastly different results based on how you ask

**Philosophy connection**: Like Socratic questioning - how you ask determines what you discover

**Basic Pattern**:
```
You are a [role]
Your task: [clear instruction]
Format: [how to structure the answer]
```

### 🔍 Demonstration: Weak vs Strong Prompts

In [ ]:
# 🔍 DEMONSTRATION: Prompt engineering with philosophical argument

def prompt_engineering_demo():
    """Show the power of good prompt design"""

    # Philosophical argument to analyze
    argument = "If we have free will, then we are morally responsible for our actions. But if determinism is true, then we don't have free will. Therefore, if determinism is true, we are not morally responsible."

    print("🔍 PROMPT ENGINEERING DEMONSTRATION")
    print(f"Argument: {argument}")
    print("=" * 70)

    # 🚫 WEAK PROMPT
    weak_prompt = f"What do you think about this argument? {argument}"

    print("\n🚫 WEAK PROMPT:")
    print(f"'{weak_prompt}'")
    print("\n📡 Sending to API...")

    weak_response = call_openrouter_api(weak_prompt)
    print(f"\nWeak Response: {weak_response}")

    time.sleep(3)

    # ✅ STRONG PROMPT
    strong_prompt = f"""You are a philosophy professor analyzing logical arguments.

Your task: Analyze this philosophical argument for logical structure and validity.

Argument: {argument}

Format your response as:
1. Premise identification (list each premise clearly)
2. Conclusion identification
3. Logical structure (valid/invalid and why)
4. One potential objection

Keep each section concise but thorough (max 50 words per section)."""

    print("\n" + "=" * 70)
    print("\n✅ STRONG PROMPT:")
    print(f"'{strong_prompt}'")
    print("\n📡 Sending to API...")

    strong_response = call_openrouter_api(strong_prompt)
    print(f"\nStrong Response: {strong_response}")

    # Comparison
    print("\n" + "=" * 70)
    print("\n📊 COMPARISON:")
    print(f"Weak response length: {len(weak_response.split())} words")
    print(f"Strong response length: {len(strong_response.split())} words")

    print("\n🎯 Key Improvements in Strong Prompt:")
    print("✅ Clear role definition (philosophy professor)")
    print("✅ Specific task description (analyze for logical structure)")
    print("✅ Structured output format (numbered sections)")
    print("✅ Length constraints (max 50 words per section)")
    print("✅ Specific requirements (premises, conclusion, validity, objection)")

    return weak_response, strong_response

# Run the demonstration
weak_demo, strong_demo = prompt_engineering_demo()

### 🎯 Exercise 4: Improve a Weak Prompt

**Your Task**: Take a weak prompt and transform it into a strong one for philosophical analysis

**Challenge**: Apply the principles you learned to create better philosophical analysis

In [ ]:
# 🎯 YOUR TURN: Transform weak prompt into strong prompt

def improve_philosophical_prompt():
    """TODO: Improve this weak prompt for better philosophical analysis"""

    quote = "The unexamined life is not worth living." # Socrates
    weak_prompt = f"Tell me about this quote: {quote}"

    print("🔧 PROMPT IMPROVEMENT EXERCISE")
    print(f"Quote: {quote}")
    print("=" * 50)

    print(f"\n🚫 WEAK PROMPT: '{weak_prompt}'")

    # TODO: Create your improved prompt using: Role + Task + Format + Constraints
    improved_prompt = f"""TODO: Write your improved prompt here.

Include:
- Clear role (e.g., "You are a philosophy scholar...")
- Specific task (e.g., "Analyze this quote...")
- Structured format (e.g., "Format as: 1. X, 2. Y...")
- Constraints (e.g., "Under 150 words, scholarly style")

Your improved prompt: {quote}"""

    print(f"\n✅ YOUR IMPROVED PROMPT: '{improved_prompt}'")

    # TODO: Test both prompts and compare results
    print("\n🔄 Testing both prompts...")

    # TODO: Test weak prompt
    weak_response = None  # TODO: Call API with weak_prompt
    print(f"Weak Result: {weak_response[:100]}...")

    time.sleep(3)

    # TODO: Test improved prompt
    improved_response = None  # TODO: Call API with improved_prompt
    print(f"Improved Result: {improved_response[:100]}...")

    # TODO: Compare word counts
    print(f"\nWeak: ___ words | Improved: ___ words")

    return weak_response, improved_response

# TODO: Run your exercise
weak_result, improved_result = None  # TODO: Call your function

## Part 5: Batch Processing (25 minutes)

### 📚 What is Batch Processing?

**Batch Processing** = Analyzing multiple texts systematically, one after another

**Why you need it**:
- Analyze 10, 50, or 100+ philosophical texts
- Consistent analysis across all texts
- Save time compared to manual analysis
- Keep organized records of all results

**Like**: Grading a stack of philosophy papers with the same rubric

### 🔍 Demonstration: Batch Process 3 Philosophical Quotes

In [ ]:
# 🔍 DEMONSTRATION: Batch processing philosophical quotes

def batch_processing_demo():
    """Process multiple philosophical quotes systematically"""

    # Sample philosophical quotes to process
    philosophical_quotes = [
        {
            'id': 1,
            'philosopher': 'Aristotle',
            'quote': 'The good life is one inspired by love and guided by knowledge.',
            'period': 'Ancient'
        },
        {
            'id': 2,
            'philosopher': 'Kant',
            'quote': 'Act only according to that maxim whereby you can at the same time will that it should become a universal law.',
            'period': 'Modern'
        },
        {
            'id': 3,
            'philosopher': 'Rawls',
            'quote': 'Justice is the first virtue of social institutions.',
            'period': 'Contemporary'
        }
    ]

    print("🔍 BATCH PROCESSING DEMONSTRATION")
    print(f"Processing {len(philosophical_quotes)} philosophical quotes...")
    print("=" * 60)

    # Create our analysis template
    analysis_template = """You are a philosophy professor analyzing philosophical quotes.

Analyze this quote by {philosopher} ({period} period): "{quote}"

Provide:
1. Main philosophical concept (1-2 sentences)
2. Ethical framework or school (1-2 sentences)
3. Contemporary relevance (1-2 sentences)

Keep response under 80 words, scholarly but accessible."""

    # Process each quote
    batch_results = []

    for i, quote_data in enumerate(philosophical_quotes):
        print(f"\n📝 Processing {i+1}/{len(philosophical_quotes)}: {quote_data['philosopher']}...")

        # Create the specific prompt for this quote
        formatted_prompt = analysis_template.format(
            philosopher=quote_data['philosopher'],
            period=quote_data['period'],
            quote=quote_data['quote']
        )

        # Send to API
        start_time = time.time()
        analysis = call_openrouter_api(formatted_prompt)
        processing_time = time.time() - start_time

        # Store the result
        result = {
            'id': quote_data['id'],
            'philosopher': quote_data['philosopher'],
            'original_quote': quote_data['quote'],
            'period': quote_data['period'],
            'analysis': analysis,
            'processing_time': processing_time,
            'success': not analysis.startswith('❌')
        }

        batch_results.append(result)

        # Show progress
        if result['success']:
            print(f"✅ Success ({processing_time:.1f}s): {analysis[:60]}...")
        else:
            print(f"❌ Failed: {analysis}")

        # Rate limiting - be nice to APIs
        time.sleep(2)

    # Create summary
    print("\n📊 BATCH PROCESSING SUMMARY:")
    print("=" * 60)
    successful = sum(1 for r in batch_results if r['success'])
    total = len(batch_results)
    avg_time = sum(r['processing_time'] for r in batch_results if r['success']) / successful if successful > 0 else 0

    print(f"Successfully processed: {successful}/{total} ({successful/total*100:.1f}%)")
    print(f"Average processing time: {avg_time:.2f} seconds")
    print(f"Total time: {sum(r['processing_time'] for r in batch_results):.1f} seconds")

    # Show a sample result
    if successful > 0:
        sample = batch_results[0]
        print(f"\n📝 Sample Analysis:")
        print(f"Philosopher: {sample['philosopher']}")
        print(f"Quote: {sample['original_quote'][:50]}...")
        print(f"Analysis: {sample['analysis']}")

    # Save results to CSV
    import pandas as pd
    results_df = pd.DataFrame(batch_results)
    results_df.to_csv('batch_philosophy_analysis.csv', index=False)
    print(f"\n💾 Results saved to 'batch_philosophy_analysis.csv'")

    print("\n💡 Key Benefits of Batch Processing:")
    print("✅ Consistent analysis across all texts")
    print("✅ Organized results with all data preserved")
    print("✅ Efficient processing of multiple texts")
    print("✅ Easy to export and analyze results")

    return batch_results

# Run the demonstration
demo_batch_results = batch_processing_demo()

### 🎯 Exercise 5: Batch Process Your Own Philosophical Texts

**Your Task**: Create your own set of philosophical texts and batch process them

**Goal**: Apply batch processing to texts relevant to your research area

In [ ]:
# 🎯 YOUR TURN: Batch process your philosophical texts

def your_batch_processing():
    """TODO: Create and process your own philosophical texts"""

    # TODO: Replace with your actual philosophical texts (4-5 texts)
    your_philosophical_texts = [
        {
            'id': 1,
            'source': 'Your Source 1',  # TODO: Real source name
            'text': 'Your philosophical text here...',  # TODO: Real text/quote
            'topic': 'ethics',  # TODO: Your topic
            'relevance': 'High'
        },
        # TODO: Add 3-4 more texts following the same structure
    ]

    print(f"🎯 Processing {len(your_philosophical_texts)} texts...")

    # TODO: Create your analysis template
    your_analysis_template = """You are a philosophy researcher.

Analyze this text from {source} (topic: {topic}): "{text}"

Provide:
1. Main claim: (2-3 sentences)
2. Approach: (1-2 sentences)
3. Research relevance: (2-3 sentences)

Under 100 words."""

    your_results = []

    # TODO: Process each text
    for i, text_data in enumerate(your_philosophical_texts):
        print(f"\n📝 Processing {i+1}: {text_data['source']}...")

        # TODO: Format prompt with text data
        formatted_prompt = None  # TODO: Use .format() to fill template

        # TODO: Send to API
        analysis = None  # TODO: Call API with formatted_prompt

        # TODO: Store result
        result = {
            'source': text_data['source'],
            'topic': text_data['topic'],
            'analysis': analysis,
            'success': not analysis.startswith('❌')
        }
        your_results.append(result)

        # TODO: Show progress
        print(f"✅ {analysis[:50]}..." if result['success'] else f"❌ Failed")
        time.sleep(2)

    # TODO: Print summary
    successful = sum(1 for r in your_results if r['success'])
    print(f"\n📊 Processed: {successful}/{len(your_results)} texts")

    return your_results

# TODO: Run your batch processing
my_results = None  # TODO: Call your function

Part 6: Real Data Upload (20 minutes) - BONUS
📚 Working with Real Research Data
Real research scenario: You have a CSV file with philosophical texts to analyze
* Paper abstracts from your literature review
* Quotes from primary sources
* Survey responses about ethical dilemmas
Today: Learn to import real data and apply all your API skills
🎯 Exercise 6: Upload and Process Your Real Data
Your Mission: Apply ALL your API skills to real philosophical data
Skills to Apply:
* ✅ API calls (Parts 1-2)
* ✅ Model comparison (Part 3)
* ✅ Prompt engineering (Part 4)
* ✅ Batch processing (Part 5)

In [ ]:
# 🎯 YOUR TURN: Complete research pipeline

def your_complete_pipeline():
    """TODO: Apply all API skills to your data"""

    print("🎯 YOUR COMPLETE RESEARCH PIPELINE")
    # Upload your own CSV file (recommended)
    # STEP 1: Import your data
    # Option A: Upload CSV file
    # from google.colab import files
    # uploaded = files.upload()
    # your_df = pd.read_csv(list(uploaded.keys())[0])

    print(f"📊 Data loaded: {your_df.shape}")

    # STEP 2: Apply your skills
    # TODO: pick a model (from Part 3)
    # TODO: Use prompt engineering (from Part 4)
    # TODO: Use batch processing (from Part 5)

    # STEP 3: Save enriched results
    # TODO: Save your results to CSV

    return your_df

# TODO: Run your complete pipeline
my_pipeline_results = your_complete_pipeline()

print("\n🎉 BONUS COMPLETE!")
print("You've applied all API skills to real philosophical data!")

## Lab 09 Summary & Next Steps

### 🎉 What You've Accomplished

✅ **API Fundamentals**: Set up secure access to HuggingFace and OpenRouter APIs
✅ **Model Comparison**: Compared free vs paid models for philosophical research
✅ **Prompt Engineering**: Learned to design effective prompts for better analysis
✅ **Data Processing**: Imported data and batch processed it through APIs

### 🛠️ Your New Research Toolkit

You now have working code for:
- Secure API access and key management
- Systematic model comparison
- Structured prompt templates for philosophical analysis
- Batch processing pipelines for multiple texts

### 🔄 From Lab 08 to Lab 09: Your Progress

| Capability | Lab 08 (Local) | Lab 09 (APIs) |
|------------|----------------|----------------|
| **Analysis Power** | Basic classification, simple Q&A | Sophisticated reasoning, nuanced analysis |
| **Flexibility** | Pre-trained tasks | Custom prompts for any philosophical task |
| **Scale** | Single texts, limited by hardware | Batch processing, cloud-scale analysis |
| **Cost** | Free | Strategic: free for exploration, paid for final work |

### 🚀 Preview: Lab 10 - RAG Systems

**Next week**: Combine your API skills with your philosophical library

**RAG (Retrieval-Augmented Generation)** will let you:
- Ask questions across your entire dissertation corpus
- Get answers with specific citations from your sources
- Find connections between different philosophical texts
- Build a "research assistant" that knows your specific materials

**Example**: "What do Kant and Rawls have in common regarding justice?" → Get answer with exact quotes and page numbers




## 🆘 Troubleshooting Guide

| Issue | Likely Cause | Solution |
|-------|--------------|----------|
| **API key not working** | Incorrect key or format | Regenerate key, check for extra spaces |
| **"Rate limit exceeded"** | Too many requests too fast | Add longer `time.sleep()` between calls |
| **"Model not found"** | Wrong model name | Check exact model names on provider website |
| **High unexpected costs** | Using expensive model by mistake | Double-check model names, use free models first |
| **Poor analysis results** | Weak prompt design | Improve prompt structure, add examples |
| **Batch processing fails** | Data format issues | Check your CSV columns, handle missing data |
| **"JSON decode error"** | API response issues | Check API status, try simpler prompts |
| **Import errors** | Wrong file path/format | Verify file location, check pandas documentation |
